In [1]:
import numpy as np
import cv2
from PIL import Image, ImageOps
import os

In [2]:
#ฟังก์ชันการนำภาพไปซ้อนบฃนอีกภาพ
def augmented_image(frame,im_src, pts_src, pts_dst):
    
    h, status = cv2.findHomography(pts_src, pts_dst)

    warped_image = cv2.warpPerspective(im_src, h, (frame.shape[1],frame.shape[0]))
 
    mask = np.zeros([frame.shape[0], frame.shape[1]], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(pts_dst), (255, 255, 255), cv2.LINE_AA)
    im_out = cv2.add(frame, warped_image, mask=cv2.bitwise_not(mask))
    im_out = cv2.add(im_out, warped_image)
    
    return im_out

In [3]:
#โหลดค่า k และ dist ที่ได้จากการ Calibration เพื่อนำมาใช้ต่อ
params_dir = os.getcwd()+'/camera_params/monocular_camera_params/'
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

In [4]:
#ฟังก์ชั่นตัวอ่าน aruco จากภาพ
AruCo_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_1000)
AruCo_params = cv2.aruco.DetectorParameters_create()

In [5]:
#ฟังก์ชันกรเขียน text ลงไปบนถาพ
def write_text(img, pose, dy, text) :
    x0 = pose[0]
    y0 = pose[1]
    for i, line in enumerate(text.split('\n')) :
        y = y0 + i*dy
        cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_DUPLEX, 0.6, (25,200,20), 2)

In [6]:
cap = cv2.VideoCapture('./videos/final_exam/Dataset-1/left_output-1.avi')

template_img = cv2.imread('./images/final_exam/Templates/Template-1.png')
template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)

chessborad = cv2.imread('./images/128.jpg')

while cap.isOpened() :
    ret, frame = cap.read()

    sharpen = np.array([[0, 0, 0],
                        [0,1.668, 0],
                        [0, 0, 0]])

    if ret :
        query_img = frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        query_gray = cv2.filter2D(gray,-1,sharpen)

        sift = cv2.SIFT_create()
        bf = cv2.BFMatcher()

        template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
        query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
        matches = bf.knnMatch(template_desc, query_desc, k=2)
        good_matches = list()
        good_matches_list = list()

        for m, n in matches :
            if m.distance < 0.7*n.distance :
                good_matches.append(m)
                good_matches_list.append([m])

        if len(good_matches) > 14.5 :

            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,1.5) 
            h, w = template_img.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)

            detected_img = cv2.polylines(frame,[np.int32(transformed_box)], True, (0,0,255), 2, cv2.LINE_AA)
            drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
        
            im_src_size = chessborad.shape[:2]
            background = np.zeros(frame.shape)
            src_points = np.float32([[0,0], [im_src_size[1],0],[im_src_size[1], im_src_size[0]] ,[0, im_src_size[0]] ])
            augmented = augmented_image(detected_img,chessborad, src_points, transformed_box)

        img = augmented.copy()
        
        markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(augmented, AruCo_dict, parameters = AruCo_params)
        
        if len(markerCorners) > 0:
            img = cv2.aruco.drawDetectedMarkers(augmented, markerCorners)
            rvecs, tvecs, points = cv2.aruco.estimatePoseSingleMarkers(markerCorners , 0.1, K, dist)
            for (rvec, tvec, id, corner) in zip(rvecs, tvecs, markerIds, markerCorners) :
                img = cv2.aruco.drawAxis(augmented, K, dist, rvec, tvec, 0.1)
                x = tvec[0,0]
                y = tvec[0,1]
                z = tvec[0,2]
                text = "x : {:.1f}\ny : {:.1f}\nz : {:.1f}".format(x, y, z)
                cX = (corner[0,0][0] + corner[0,2][0]) / 2
                cY = (corner[0,0][1] + corner[0,2][1]) / 2.1
                write_text(detected_img, (cX, cY), 20, text)
                write_text(img, (cX, cY), 20, text)
            
        cv2.imshow('Final_exam',detected_img)
        
        if cv2.waitKey(int(1000/44)) & 0xFF == ord('q') : 
            break
    else :
        break
    
cv2.destroyAllWindows()
cap.release()